In [ ]:
from os import system
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers import Attention
import numpy as np
import sys
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from pickle import dump
from google.colab import drive
drive.mount('/content/drive')

np.set_printoptions(threshold=sys.maxsize)
# from scipy.signal import argrelextrema
# from statsmodels.nonparametric.kernel_regression import KernelReg
def find_MaxtimeStep(df):
    x = df.loc[df['TREND'].notnull()].index.tolist()
    max_distance =0
    for i in range (0,len(x)-1):
        if x[i+1]-x[i] >max_distance:
            max_distance = x[i+1]-x[i]
    return max_distance
def get_fixedSizeCandle(candles,md):
    candles_array = np.array(candles)
    Fixed_MaxSizeCandles = np.zeros((md,candles_array.shape[1]),dtype=float)
    Fixed_MaxSizeCandles[md-candles_array.shape[0]:,:candles_array.shape[1]] =candles_array
    return Fixed_MaxSizeCandles

def PreparetestData(path,Max_TimeStep):
    print ('Reading data...')
    excel_data = pd.read_excel(path)
    df = pd.DataFrame(excel_data, columns=['DATE', 'TIME', 'OPEN','HIGH','LOW','CLOSE','TREND'])
    print ('Reading data Complete!')
    print ('Creating Input Array ...')

    x=list()
    y=list()
    i=0
    candles = list()
    while i<len(df):
        candle =list()
        candle = np.zeros(4)
        if not(pd.isnull(df.at[i,'TREND'])):
            y.append(df.at[i,'TREND'])
            if len(candles)>0:
                    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
            candles = list()
        else:
          for j in range (0,4):
              candle[j] = df.iloc[i,j+2] -df.iloc[i-1,j+2]
            #candle[j] = df.iloc[i,j+2]
        candles.append(candle)
        i = i+1
    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
    x = np.array(x)
    y = np.array(y)
    scaler = StandardScaler()
    for i in range(0,x.shape[0]):
        x[i]= scaler.fit_transform(x[i])
    #x = scaler.fit_transform(x.reshape(-1, x.shape[-1])).reshape(x.shape)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(y)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    print ('Creating Input Array Complete!')
    return np.array(x),np.array(onehot_encoded)

def PrepareData(path):
    print ('Reading data...')
    excel_data = pd.read_excel(path)
    df = pd.DataFrame(excel_data, columns=['DATE', 'TIME', 'OPEN','HIGH','LOW','CLOSE','TREND'])
    print ('Reading data Complete!')
    print ('Creating Input Array ...')

    Max_TimeStep = find_MaxtimeStep(df)
    x=list()
    y=list()
    i=0
    candles = list()
    while i<len(df):
        candle =list()
        candle = np.zeros(4)
        if not(pd.isnull(df.at[i,'TREND'])):
            y.append(df.at[i,'TREND'])
            if len(candles)>0:
                    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
            candles = list()
        else:
          for j in range (0,4):
              candle[j] = df.iloc[i,j+2] -df.iloc[i-1,j+2]
            #candle[j] = df.iloc[i,j+2]
        candles.append(candle)
        i = i+1

    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
    x = np.array(x)
    y = np.array(y)
    scaler = StandardScaler()
    for i in range(0,x.shape[0]):
      x[i]= scaler.fit_transform(x[i])
    #x = scaler.fit_transform(x.reshape(-1, x.shape[-1])).reshape(x.shape)
    #dump(scaler, open('/content/drive/MyDrive/Dataset/std_scaler.bin', 'wb'))
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(y)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    print ('Creating Input Array Complete!')
    return np.array(x),np.array(onehot_encoded),Max_TimeStep

def Create_Model(num_timesteps,num_features):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(num_timesteps, num_features)))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dense(32))
    model.add(Dense(2,activation= 'softmax'))
    return model

def Create_Train_Test(x,y,test_Persent):
    X_Test = x[0:int((test_Persent)*x.shape[0]/100),:,:]
    X_Train = x[(X_Test.shape[0]):,:,:]
    Y_Test= y[0:X_Test.shape[0],:]
    Y_Train = y[X_Test.shape[0]:,:]
    print(X_Test.shape)
    print(X_Train.shape)
    print(Y_Test.shape)
    print(Y_Train.shape)
    return X_Train,X_Test,Y_Train,Y_Test

#set parameters
test_Persent = 10
Max_TimeStep=0
Max_TimeStep1=0
print('Preparing Data ...')
x,y,Max_TimeStep = PrepareData('/content/drive/MyDrive/Dataset/DataSet_Up_Down.xlsx')
#x1,y1, = PreparetestData('/content/drive/MyDrive/Test_Sample.xlsx',Max_TimeStep)
X_Train,X_Test,Y_Train,Y_Test = Create_Train_Test(x,y,test_Persent)

print('Data is Ready ... ')
print( x.shape,Max_TimeStep)

model = Create_Model(Max_TimeStep,X_Train.shape[2])
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
history= model.fit(X_Train, Y_Train, epochs=2,  verbose=2)
model.save('/content/drive/MyDrive/Dataset/8_31_UD_2Epoch_Model')
scores =model.evaluate(X_Test,Y_Test,verbose=0)
print(scores)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Preparing Data ...
Reading data...
Reading data Complete!
Creating Input Array ...
Creating Input Array Complete!
(128, 500, 4)
(1152, 500, 4)
(128, 2)
(1152, 2)
Data is Ready ... 
(1280, 500, 4) 500
Epoch 1/2
36/36 - 35s - loss: 0.1519 - accuracy: 0.9505 - 35s/epoch - 978ms/step
Epoch 2/2
36/36 - 31s - loss: 0.0048 - accuracy: 0.9991 - 31s/epoch - 858ms/step


[0.00021311394812073559, 1.0]
